In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [6]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import copy

# ==========================================
# 1. CONFIGURATION
# ==========================================
BASE_DIR = '/kaggle/input/dlp-image-classification-for-handwriting/downlaod1'
TRAIN_CSV_PATH = os.path.join(BASE_DIR, 'train.csv')
TEST_DIR = os.path.join(BASE_DIR, 'test')
SAMPLE_SUB_PATH = os.path.join(BASE_DIR, 'sample_submission.csv')
TRAIN_DIR = os.path.join(BASE_DIR, 'train') 

# High Resolution & EfficientNet B1
IMG_SIZE = 256 
BATCH_SIZE = 32
LEARNING_RATE = 0.0005 
EPOCHS = 15
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE} | Model: EfficientNet B1 | Res: {IMG_SIZE}x{IMG_SIZE}")

# ==========================================
# 2. DATASET
# ==========================================
class HandwritingDataset(Dataset):
    def __init__(self, df, img_dir, transform=None, is_test=False):
        self.data = df
        self.img_dir = img_dir
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError:
            image = Image.open(os.path.join(BASE_DIR, img_name)).convert("RGB")

        if self.transform:
            image = self.transform(image)

        if self.is_test:
            return image, img_name
        else:
            label = int(self.data.iloc[idx, 1])
            return image, label

# ==========================================
# 3. AUGMENTATION
# ==========================================
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        # Random rotations and shifts to mimic handwriting variance
        transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.85, 1.15)), 
        transforms.ColorJitter(brightness=0.1, contrast=0.1), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# ==========================================
# 4. PREPARE DATA (Fixed Split)
# ==========================================
full_train_df = pd.read_csv(TRAIN_CSV_PATH)

# Use exactly 20 images for validation. 
# This ensures we have enough slots for all 10 classes (0-9).
train_df, val_df = train_test_split(full_train_df, test_size=20, stratify=full_train_df['label'], random_state=42)

print(f"Training on {len(train_df)} samples. Validation on {len(val_df)} samples.")

train_ds = HandwritingDataset(train_df, TRAIN_DIR, transform=data_transforms['train'])
val_ds = HandwritingDataset(val_df, TRAIN_DIR, transform=data_transforms['val'])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# ==========================================
# 5. MODEL: EFFICIENTNET B1
# ==========================================
# We use the 'models' module imported at the top to avoid import syntax errors
model = models.efficientnet_b1(weights=models.EfficientNet_B1_Weights.DEFAULT)

# Adjust Classifier for 10 classes
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 10)
model = model.to(DEVICE)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=EPOCHS, T_mult=1)

# ==========================================
# 6. TRAINING LOOP
# ==========================================
def train_model(model, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        # Training Phase
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
        for inputs, labels in pbar:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            
            pbar.set_postfix({'loss': loss.item()})
        
        scheduler.step()
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct / total
        
        print(f"Epoch {epoch+1} | Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")

    print("Training Complete.")
    return model

# Train
model = train_model(model, EPOCHS)

# ==========================================
# 7. SUBMISSION
# ==========================================
sample_sub = pd.read_csv(SAMPLE_SUB_PATH)
test_ds = HandwritingDataset(sample_sub, TEST_DIR, transform=data_transforms['val'], is_test=True)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("\nGenerating predictions...")
model.eval()

submission_rows = []
with torch.no_grad():
    for inputs, img_names in tqdm(test_loader):
        inputs = inputs.to(DEVICE)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        preds_list = preds.cpu().numpy()
        
        for img_name, pred in zip(img_names, preds_list):
            submission_rows.append({'image_id': img_name, 'label': pred})

submission_df = pd.DataFrame(submission_rows)
submission_df.to_csv('submission.csv', index=False)
print("Saved 'submission_b1_fixed.csv'")

Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth


Using device: cuda | Model: EfficientNet B1 | Res: 256x256
Training on 910 samples. Validation on 20 samples.


100%|██████████| 30.1M/30.1M [00:00<00:00, 193MB/s]


Epoch 1 | Train Loss: 2.0342 | Train Acc: 0.3473


Epoch 2 | Train Loss: 0.9204 | Train Acc: 0.8989


...

Epoch 13 | Train Loss: 0.5204 | Train Acc: 0.9967


Epoch 14 | Train Loss: 0.5166 | Train Acc: 0.9978


Epoch 15 | Train Loss: 0.5189 | Train Acc: 0.9956
Training Complete.

Generating predictions...


100%|██████████| 13/13 [00:01<00:00, 10.89it/s]

Saved 'submission_b1_fixed.csv'
